# Lab 7 (Advanced): Multi-Router Hybrid AI with Azure AI Foundry Agents

**Purpose:** Build an advanced Streamlit interface demonstrating multiple routing strategies with Azure AI Foundry Agents for cloud processing.

## Overview

This advanced lab demonstrates:
- Multiple routing strategies (Rule-based, BERT, Phi SLM)
- Azure AI Foundry Agents for sophisticated cloud processing
- Side-by-side router comparison
- Advanced telemetry and performance analysis
- Production-ready hybrid architecture

## Step 7A.1: Import Dependencies and Configuration

In [ ]:
import os
import sys
import time
import json
import warnings
from typing import Dict, List, Tuple, Optional, Any
from datetime import datetime
from dotenv import load_dotenv

# Suppress warnings
warnings.filterwarnings('ignore')

# Add parent directory for module imports
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Load environment
load_dotenv()

print("✅ Base dependencies loaded")

In [ ]:
# Import Azure AI Foundry Agents
try:
    from azure.ai.projects import AIProjectClient
    from azure.ai.agents.models import CodeInterpreterTool
    from azure.identity import DefaultAzureCredential
    foundry_agents_available = True
    print("✅ Azure AI Foundry Agents SDK available")
except ImportError as e:
    foundry_agents_available = False
    print(f"⚠️ Azure AI Foundry Agents not available: {e}")

# Import OpenAI clients
from openai import OpenAI, AzureOpenAI

# Import local foundry manager
from foundry_local import FoundryLocalManager

# Import routing modules
try:
    from modules.router import analyze_query_characteristics, route_query
    from modules.bert_router import BertQueryRouter, BertRouterConfig
    from modules.phi_router import PhiQueryRouter, PhiRouterConfig
    routing_modules_available = True
    print("✅ All routing modules imported")
except ImportError as e:
    routing_modules_available = False
    print(f"⚠️ Some routing modules not available: {e}")

print(f"Configuration status:")
print(f"  - Foundry Agents: {foundry_agents_available}")
print(f"  - Routing Modules: {routing_modules_available}")

## Step 7A.2: Initialize Local and Cloud Clients

In [ ]:
# Initialize local foundry manager
manager = FoundryLocalManager(alias_or_model_id=None, bootstrap=True)
LOCAL_ENDPOINT = manager.service_uri
LOCAL_MODEL_NAME = "Phi-3.5-mini-instruct-generic-cpu"  # Fixed model name

print(f"Local service: {LOCAL_ENDPOINT}")
print(f"Local model: {LOCAL_MODEL_NAME}")

In [ ]:
def initialize_advanced_clients():
    """Initialize all clients: local, Azure OpenAI, and Azure AI Foundry Agents."""
    clients = {
        'local': None,
        'azure_openai': None, 
        'azure_agent': None,
        'status': {}
    }
    
    # Local OpenAI client
    try:
        clients['local'] = OpenAI(
            base_url=f"{LOCAL_ENDPOINT}/v1",
            api_key="not-needed"
        )
        clients['local_model'] = LOCAL_MODEL_NAME
        clients['status']['local'] = '✅ Ready'
        print(f"✅ Local client: {LOCAL_MODEL_NAME}")
    except Exception as e:
        clients['status']['local'] = f'❌ Failed: {e}'
        print(f"❌ Local client failed: {e}")
    
    # Azure OpenAI client (fallback)
    try:
        clients['azure_openai'] = AzureOpenAI(
            api_key=os.environ["AZURE_OPENAI_KEY"],
            api_version=os.environ["AZURE_OPENAI_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
        )
        clients['azure_model'] = os.environ["AZURE_DEPLOYMENT_NAME"]
        clients['status']['azure_openai'] = '✅ Ready'
        print(f"✅ Azure OpenAI: {clients['azure_model']}")
    except Exception as e:
        clients['status']['azure_openai'] = f'❌ Failed: {e}'
        print(f"❌ Azure OpenAI failed: {e}")
    
    # Azure AI Foundry Agent client
    try:
        if foundry_agents_available:
            project_client = AIProjectClient(
                endpoint=os.environ["AZURE_AI_FOUNDRY_ENDPOINT"],
                credential=DefaultAzureCredential()
            )
            clients['azure_agent'] = project_client
            clients['status']['azure_agent'] = '✅ Ready'
            print("✅ Azure AI Foundry Agent client ready")
        else:
            clients['status']['azure_agent'] = '⚠️ SDK not available'
    except Exception as e:
        clients['status']['azure_agent'] = f'❌ Failed: {e}'
        print(f"❌ Azure AI Foundry Agent failed: {e}")
    
    return clients

# Initialize all clients
clients = initialize_advanced_clients()
print(f"\n📊 Client Status Summary:")
for name, status in clients['status'].items():
    print(f"  {name}: {status}")

## Step 7A.3: Initialize Multiple Routers

In [ ]:
# Import specialized routers from modules
try:
    from modules.bert_router import BertQueryRouter, BertRouterConfig
    bert_available = True
    print("✅ BERT router module imported")
except ImportError as e:
    bert_available = False
    print(f"⚠️ BERT router not available: {e}")


routers = {
'rule_based': True,
'bert': None,
'phi': None,
'status': {}
}

# Rule-based router (always available)
routers['status']['rule_based'] = '✅ Ready'
print("✅ Rule-based router ready")

bert_config = BertRouterConfig(
            model_path="./mobilbert_query_router_trained",
            confidence_threshold=0.7
)
bert_router = BertQueryRouter(bert_config)
bert_router._load_model()
if bert_available:
    routers['bert'] = bert_router
    routers['status']['bert'] = '✅ Ready'
    print("✅ BERT router loaded")
else:
    routers['status']['bert'] = '⚠️ Model not found'
    print("⚠️ BERT model not found")

In [ ]:
def initialize_routers():
    """Initialize all available routing strategies."""
    routers = {
        'rule_based': True,
        'bert': None,
        'phi': None,
        'status': {}
    }
    
    # Rule-based router (always available)
    routers['status']['rule_based'] = '✅ Ready'
    print("✅ Rule-based router ready")
    
    # BERT router
    # Import specialized routers from modules
    try:
        from modules.bert_router import BertQueryRouter, BertRouterConfig
        bert_available = True
        print("✅ BERT router module imported")
    except ImportError as e:
        bert_available = False
        print(f"⚠️ BERT router not available: {e}")

    try:
        if routing_modules_available:
            bert_config = BertRouterConfig()
            bert_router = BertQueryRouter(bert_config)
            bert_router._load_model()
            if bert_available:
                routers['bert'] = bert_router
                routers['status']['bert'] = '✅ Ready'
                print("✅ BERT router loaded")
            else:
                routers['status']['bert'] = '⚠️ Model not found'
                print("⚠️ BERT model not found")
        else:
            routers['status']['bert'] = '❌ Module unavailable'
    except Exception as e:
        routers['status']['bert'] = f'❌ Failed: {str(e)[:50]}'
        print(f"❌ BERT router failed: {e}")
    
    # Phi router
    try:
        if routing_modules_available:
            phi_config = PhiRouterConfig(model_path="./phi_router_model")
            phi_router = PhiQueryRouter(phi_config)
            if phi_router.load_model():
                routers['phi'] = phi_router
                routers['status']['phi'] = '✅ Ready'
                print("✅ Phi router loaded")
            else:
                routers['status']['phi'] = '⚠️ Model not found'
                print("⚠️ Phi model not found")
        else:
            routers['status']['phi'] = '❌ Module unavailable'
    except Exception as e:
        routers['status']['phi'] = f'❌ Failed: {str(e)[:50]}'
        print(f"❌ Phi router failed: {e}")
    
    return routers

# Initialize routers
routers = initialize_routers()
print(f"\n📊 Router Status Summary:")
for name, status in routers['status'].items():
    print(f"  {name}: {status}")

## Step 7A.4: Create Advanced Routing Functions

In [ ]:
def route_with_strategy(query: str, strategy: str = 'rule_based') -> Tuple[str, str, float]:
    """Route query using specified strategy."""
    start_time = time.time()
    
    try:
        if strategy == 'rule_based':
            analysis = analyze_query_characteristics(query)
            target, reason = route_query(query, analysis)
            confidence = 0.85  # Default confidence
            
        elif strategy == 'bert' and routers['bert']:
            result = routers['bert'].route_query(query)
            target = result['prediction']
            reason = f"BERT: {result['reasoning']}"
            confidence = result['confidence']
            
        elif strategy == 'phi' and routers['phi']:
            result = routers['phi'].route_query(query)
            target = result['prediction']
            reason = f"Phi: {result['reasoning']}"
            confidence = result['confidence']
            
        else:
            # Fallback to rule-based
            analysis = analyze_query_characteristics(query)
            target, reason = route_query(query, analysis)
            confidence = 0.75
            reason = f"Fallback: {reason}"
    
    except Exception as e:
        target, reason, confidence = 'cloud', f'Error routing: {e}', 0.5
    
    routing_time = time.time() - start_time
    return target, reason, confidence

def generate_response_with_foundry(query: str, chat_history: List = None) -> Tuple[str, float]:
    """Generate response using Azure AI Foundry Agents."""
    if chat_history is None:
        chat_history = []
    
    start_time = time.time()
    
    try:
        if clients['azure_agent']:
            # Create agent for complex processing
            agent = clients['azure_agent'].agents.create_agent(
                model=clients['azure_model'],
                name="hybrid-assistant",
                instructions="You are a helpful AI assistant with access to advanced reasoning capabilities.",
                tools=[CodeInterpreterTool()]
            )
            
            # Create thread and get response
            thread = clients['azure_agent'].agents.threads.create()
            
            # Add message to thread
            clients['azure_agent'].agents.messages.create(
                thread_id=thread.id,
                role="user",
                content=query
            )
            
            # Run agent
            run = clients['azure_agent'].agents.runs.create_and_process(
                thread_id=thread.id,
                agent_id=agent.id
            )
            
            # Wait for completion
            while run.status in ['queued', 'in_progress']:
                time.sleep(1)
                run = clients['azure_agent'].agents.runs.get(thread.id, run.id)
            
            # Get messages
            messages = clients['azure_agent'].agents.messages.list(thread.id)
            response = messages.data[0].content[0].text.value
            
            # Cleanup
            clients['azure_agent'].agents.delete_agent(agent.id)
            clients['azure_agent'].agents.delete_thread(thread.id)
            
        else:
            # Fallback to Azure OpenAI
            messages = chat_history + [{"role": "user", "content": query}]
            response_obj = clients['azure_openai'].chat.completions.create(
                model=clients['azure_model'],
                messages=messages,
                max_tokens=400,
                temperature=0.7
            )
            response = response_obj.choices[0].message.content
            
    except Exception as e:
        response = f"Error generating response: {e}"
    
    response_time = time.time() - start_time
    return response, response_time

def generate_multi_router_response(query: str, chat_history: List = None) -> Dict[str, Any]:
    """Generate response using multiple routing strategies for comparison."""
    if chat_history is None:
        chat_history = []
    
    results = {
        'query': query,
        'timestamp': datetime.now().isoformat(),
        'routing_results': {},
        'final_response': None,
        'response_time': 0,
        'source': None
    }
    
    # Test all available routers
    strategies = ['rule_based', 'bert', 'phi']
    for strategy in strategies:
        target, reason, confidence = route_with_strategy(query, strategy)
        results['routing_results'][strategy] = {
            'target': target,
            'reason': reason,
            'confidence': confidence,
            'available': routers['status'].get(strategy, '❌').startswith('✅')
        }
    
    # Use primary strategy (rule_based) for actual response
    primary_target = results['routing_results']['rule_based']['target']
    
    start_time = time.time()
    
    try:
        if primary_target == 'local' and clients['local']:
            messages = chat_history + [{"role": "user", "content": query}]
            response_obj = clients['local'].chat.completions.create(
                model=clients['local_model'],
                messages=messages,
                max_tokens=200,
                temperature=0.7
            )
            results['final_response'] = response_obj.choices[0].message.content
            results['source'] = 'Local'
            
        else:
            # Use Azure AI Foundry Agents for cloud processing
            response, _ = generate_response_with_foundry(query, chat_history)
            results['final_response'] = response
            results['source'] = 'Azure AI Foundry' if clients['azure_agent'] else 'Azure OpenAI'
            
    except Exception as e:
        results['final_response'] = f"Error: {e}"
        results['source'] = 'Error'
    
    results['response_time'] = time.time() - start_time
    return results

print("✅ Advanced routing functions created")

## Step 7A.5: Test Multi-Router System

In [ ]:
# Test the multi-router system
test_queries = [
    "Hello!",
    "What is 42 * 7?",
    "Analyze the impact of hybrid AI architectures on enterprise productivity"
]

print("🧪 Testing Multi-Router System:")
print("=" * 60)

for i, query in enumerate(test_queries, 1):
    print(f"\n{i}. Query: {query}")
    print("-" * 40)
    
    result = generate_multi_router_response(query)
    
    # Show routing decisions
    print("Router Decisions:")
    for strategy, data in result['routing_results'].items():
        status = "✅" if data['available'] else "❌"
        print(f"  {status} {strategy}: {data['target']} (conf: {data['confidence']:.2f})")
    
    # Show response
    print(f"\n📝 Response ({result['source']}):") 
    response_preview = result['final_response'][:100] + "..." if len(result['final_response']) > 100 else result['final_response']
    print(f"   {response_preview}")
    print(f"⏱️  Time: {result['response_time']:.3f}s")
    print()

print("✅ Multi-router testing complete!")

## Step 7A.6: Create Advanced Streamlit Application

## 🎉 Lab 7A Complete!

### Advanced Features Implemented:
- ✅ **Multiple Routing Strategies**: Rule-based, BERT, and Phi SLM routers
- ✅ **Azure AI Foundry Agents**: Advanced cloud processing with agent capabilities
- ✅ **Router Comparison**: Side-by-side analysis of routing decisions
- ✅ **Advanced Telemetry**: Detailed performance metrics and visualization
- ✅ **Production Architecture**: Robust error handling and fallback mechanisms

### Key Differentiators from Basic Lab 7:
1. **Multi-Router Support**: Compare different routing algorithms in real-time
2. **Azure AI Foundry**: Advanced agent-based processing for complex queries
3. **Enhanced Analytics**: Routing distribution charts and performance tracking
4. **Modular Design**: Easy to add new routing strategies
5. **Professional UI**: Advanced Streamlit interface with detailed debugging

### Demonstration Value:
Perfect for showcasing the evolution of AI routing from simple rules to advanced ML-based decisions, while leveraging Azure's most sophisticated AI services for cloud processing.

🚀 **Run**: `streamlit run app_advanced.py`